In [3]:
from tensorflow.compat.v1.keras.applications.resnet import ResNet50

In [8]:
X = tf.keras.Input(name="X", shape=(299,299,3), dtype=tf.dtypes.float32)
training = tf.keras.backend.placeholder(shape=[]) #tf.placeholder_with_default(False, shape=[])
with tf.name_scope("model"):
    model = tf.keras.applications.ResNet50V2(
    include_top=True, weights='imagenet', input_tensor=X,
    pooling=None, classes=1000, classifier_activation='softmax')


resnet_saver = tf.compat.v1.train.Saver(model)

In [4]:
from tensorflow.compat.v1.keras.applications.resnet import ResNet50
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None,224,244,3], name="X")
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    #training = tf.placeholder_with_default(False, shape=[], name='training')

with tf.name_scope("model"):
    output = ResNet50(
    include_top=True, weights='imagenet', input_tensor=X,
    pooling=None, classes=1000, classifier_activation='softmax')

# with tf.name_scope("output"):
#     logits = tf.layers.dense(fc1_drop, n_outputs, name="output")
#     Y_proba = tf.nn.softmax(logits, name="Y_proba")

# with tf.name_scope("train"):
#     xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=ouput, labels=y)
#     loss = tf.reduce_mean(xentropy)
#     optimizer = tf.train.AdamOptimizer()
#     training_op = optimizer.minimize(loss)

# with tf.name_scope("eval"):
#     correct = tf.nn.in_top_k(logits, y, 1)
#     accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

Instructions for updating:
non-resource variables are not supported in the long term
102973440/102967424 [==============================] - 8s 0us/step


In [1]:
with tf.Session() as sess:
    saver.restore(sess, "../checkpoints/resnet_v2_50.ckpt")

NameError: name 'X_batch' is not defined

True

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

AttributeError: module 'tensorflow._api.v2.train' has no attribute 'Saver'

In [17]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [11]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

height = 28
width = 28
channels = 1
n_inputs = height * width

conv1_fmaps = 32
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 64
conv2_ksize = 3
conv2_stride = 1
conv2_pad = "SAME"
conv2_dropout_rate = 0.25

pool3_fmaps = conv2_fmaps

n_fc1 = 128
fc1_dropout_rate = 0.5

n_outputs = 10


with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    training = tf.placeholder_with_default(False, shape=[], name='training')

conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, name="conv1")
conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv2")

with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 14 * 14])
    pool3_flat_drop = tf.layers.dropout(pool3_flat, conv2_dropout_rate, training=training)

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat_drop, n_fc1, activation=tf.nn.relu, name="fc1")
    fc1_drop = tf.layers.dropout(fc1, fc1_dropout_rate, training=training)

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1_drop, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.Dense instead.


In [12]:
def get_model_params():
    gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    return {gvar.op.name: value for gvar, value in zip(gvars, tf.get_default_session().run(gvars))}

def restore_model_params(model_params):
    gvar_names = list(model_params.keys())
    assign_ops = {gvar_name: tf.get_default_graph().get_operation_by_name(gvar_name + "/Assign")
                  for gvar_name in gvar_names}
    init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
    feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
    tf.get_default_session().run(assign_ops, feed_dict=feed_dict)

In [15]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [18]:
import numpy as np

n_epochs = 1000
batch_size = 50
iteration = 0

best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            iteration += 1
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
            if iteration % check_interval == 0:
                loss_val = loss.eval(feed_dict={X: X_valid, y: y_valid})
                if loss_val < best_loss_val:
                    best_loss_val = loss_val
                    checks_since_last_progress = 0
                    best_model_params = get_model_params()
                else:
                    checks_since_last_progress += 1
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print("Epoch {}, last batch accuracy: {:.4f}%, valid. accuracy: {:.4f}%, valid. best loss: {:.6f}".format(
                  epoch, acc_batch * 100, acc_val * 100, best_loss_val))
        if checks_since_last_progress > max_checks_without_progress:
            print("Early stopping!")
            break

    if best_model_params:
        restore_model_params(best_model_params)
    acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
    print("Final accuracy on test set:", acc_test)
    save_path = saver.save(sess, "./my_mnist_model")

Epoch 0, last batch accuracy: 100.0000%, valid. accuracy: 98.3400%, valid. best loss: 0.062969
Epoch 1, last batch accuracy: 98.0000%, valid. accuracy: 98.6200%, valid. best loss: 0.046129
Epoch 2, last batch accuracy: 100.0000%, valid. accuracy: 98.9000%, valid. best loss: 0.045370
Epoch 3, last batch accuracy: 100.0000%, valid. accuracy: 98.8600%, valid. best loss: 0.041171
Epoch 4, last batch accuracy: 100.0000%, valid. accuracy: 99.0200%, valid. best loss: 0.033647
Epoch 5, last batch accuracy: 100.0000%, valid. accuracy: 98.9400%, valid. best loss: 0.033647
Epoch 6, last batch accuracy: 100.0000%, valid. accuracy: 99.1600%, valid. best loss: 0.033647
Epoch 7, last batch accuracy: 100.0000%, valid. accuracy: 99.1000%, valid. best loss: 0.032423
Epoch 8, last batch accuracy: 100.0000%, valid. accuracy: 99.0800%, valid. best loss: 0.032423
Epoch 9, last batch accuracy: 100.0000%, valid. accuracy: 99.0800%, valid. best loss: 0.032423
Epoch 10, last batch accuracy: 100.0000%, valid. ac